In [1]:
import pandas as pd
import re

In [2]:
import ast #usamos para convertir valores string en columna en listas reales

In [3]:
data = pd.read_csv("recetas_cat_0301.csv")

In [4]:
df = data.copy()

### Convertimos ingredientes en listas de elementos y no cadenas

In [5]:
df["ingredients"] = df["ingredients"].apply(ast.literal_eval)

In [6]:
# Explode a ingredientes, para tener receta x ingrediente

df_ingredients = df.explode("ingredients")[["nom", "ingredients"]]

In [7]:
df_ingredients.head(50)

,nom,ingredients
0,Iogurt grec amb fraules,Iogurt grec 200g
0,Iogurt grec amb fraules,Fraules 100-150g
0,Iogurt grec amb fraules,Nous i ametlles 30g
0,Iogurt grec amb fraules,Xía 10-20g
0,Iogurt grec amb fraules,Canyella al gust
1,Solomillo de vedella Strogonoff i arròs basmati,Solomillo de vedella 150-200g
1,Solomillo de vedella Strogonoff i arròs basmati,Xampinyons 100g
1,Solomillo de vedella Strogonoff i arròs basmati,Ceba blanca 50g
1,Solomillo de vedella Strogonoff i arròs basmati,Alls 2
1,Solomillo de vedella Strogonoff i arròs basmati,Julivert fresc 3g


In [8]:
df_ingredients.to_csv("para_separar_cantidades.csv", index=False)

### Regex para obtener cantidad/unidad

In [9]:
# Función para procesar y separar los ingredientes
def process_ingredient(text):
    if pd.isna(text):  # Verificar si el valor es NaN
        return pd.Series({'ingredient': None, 'cantitat': None, 'unitat': None})

    text = str(text).strip()

    # Patrones regex para identificar cantidad, unidad y nombre del ingrediente
    patterns = [
        # Patrón para "X g/ml de Ingrediente"
        (r'^(\d+)\s*(g|ml)\s+(?:de\s+)?(.+)$', lambda m: (m.group(3), m.group(1), m.group(2))),

        # Patrón para "X cucharada(s)/cucharadita(s) de Ingrediente"
        (r'^(\d+)\s*(cucharada|cucharadita)s?\s+(?:de\s+)?(.+)$', lambda m: (m.group(3), m.group(1), m.group(2))),

        # Patrón para "X taza (Y ml) de Ingrediente"
        (r'^(\d+)\s*taza\s*$(\d+)\s*ml$\s*de\s*(.+)$', lambda m: (m.group(3), m.group(2), 'ml')),

        # Patrón para fracciones simples "1/2 Ingrediente" o "½ Ingrediente"
        (r'^(?:1/2|½)\s+(.+)$', lambda m: (m.group(1), '1/2', None)),

        # Patrón para "X diente(s) de ajo"
        (r'^(\d+)\s*dientes?\s+de\s+(.+)$', lambda m: (m.group(2), m.group(1), 'unitat')),

        # Patrón para "X ingrediente(s)"
        (r'^(\d+)\s+(.+)$', lambda m: (m.group(2), m.group(1), 'unitat')),

        # Patrón para "X ml" y "X ml de"
        (r'^(\d+)\s*ml(?:\s+de)?\s*(.+)$', lambda m: (m.group(2).strip(), m.group(1), 'ml')),

        # Patrón para "X g" y "X g de"
        #(r'^(\d+)\s*g(?:\s+de)?\s*(.+)$', lambda m: (m.group(2).strip(), m.group(1), 'grams')),
        (r"^(\d+)\s*g(?:\s+d['’]?\s+de)?\s*(.+)$", lambda m: (m.group(2).strip(), m.group(1), 'grams')),
        
        # Patrón para "texto X g"
        (r'^(.+?)\s+(\d+)\s*(g|grams|ml|mil·lilitres|kg|l|litres)$', lambda m: (m.group(1).strip(), m.group(2), m.group(3)))
    ]

    for pattern, handler in patterns:
        match = re.match(pattern, text)
        if match:
            ingredient, cantitat, unitat = handler(match)
            return pd.Series({'ingredient': ingredient.strip(), 'cantitat': cantitat, 'unitat': unitat})

    # Si no coincide con ningún patrón, devolver solo el ingrediente
    return pd.Series({'ingredient': text, 'cantitat': None, 'unitat': None})

# Aplicar la función a cada valor de la columna "ingredients"
df_ingredients[['ingredient', 'cantitat', 'unitat']] = df_ingredients['ingredients'].apply(process_ingredient)

# Limpiar y ajustar el resultado
df_ingredients['cantitat'] = df_ingredients['cantitat'].replace('', None)
df_ingredients['unitat'] = df_ingredients['unitat'].map({
    'g': 'grams',
    'ml': 'mil·lilitres',
    'cucharada': 'cullerada',
    'cucharadita': 'culleradeta',
    'unitat': 'unitat'
})

# Limpiar paréntesis y texto opcional
df_ingredients['ingredient'] = df_ingredients['ingredient'].apply(lambda x: re.sub(r'$[^)]*$', '', str(x)).strip())
df_ingredients['ingredient'] = df_ingredients['ingredient'].apply(lambda x: re.sub(r'\s+', ' ', str(x)).strip())

In [10]:
df_ingredients["ingredient"].value_counts().head(10)

ingredient
Sal i pebre al gust    56
d'oli d'oliva          25
ous                    25
Sal                    17
alls                   17
ceba blanca            16
1⁄2 llimona            16
formatge parmesà       14
mantega sense sal      13
ou                     12
Name: count, dtype: int64

#### Reemplazando ingredientes corregidos

In [11]:
df_ingredients["ingredient"] = df_ingredients["ingredient"].replace(r"d['’]oli d'oliva", "oli d'oliva", regex=True)

In [12]:
df_ingredients["ingredient"] = df_ingredients["ingredient"].replace(r"d['’]eritritol", "eritritol", regex=True)

In [13]:
df_ingredients["ingredient"] = df_ingredients["ingredient"].replace({
    "d'essència de vainilla": "essència de vainilla",
    "d'espinacs": "espinacs",
    "d'orenga fresc (0.5g en pols)": "orenga fresc (0.5g en pols)",
    "Oli d'oliva": "oli d'oliva",
    "d'ametlles": "d'ametlles"
    
})

In [14]:
df_ingredients["ingredient"].nunique()

563

In [15]:
df_ingredients.head(50)

,nom,ingredients,ingredient,cantitat,unitat
0,Iogurt grec amb fraules,Iogurt grec 200g,Iogurt grec,200,grams
0,Iogurt grec amb fraules,Fraules 100-150g,Fraules 100-150g,None,NaN
0,Iogurt grec amb fraules,Nous i ametlles 30g,Nous i ametlles,30,grams
0,Iogurt grec amb fraules,Xía 10-20g,Xía 10-20g,None,NaN
0,Iogurt grec amb fraules,Canyella al gust,Canyella al gust,None,NaN
1,Solomillo de vedella Strogonoff i arròs basmati,Solomillo de vedella 150-200g,Solomillo de vedella 150-200g,None,NaN
1,Solomillo de vedella Strogonoff i arròs basmati,Xampinyons 100g,Xampinyons,100,grams
1,Solomillo de vedella Strogonoff i arròs basmati,Ceba blanca 50g,Ceba blanca,50,grams
1,Solomillo de vedella Strogonoff i arròs basmati,Alls 2,Alls 2,None,NaN
1,Solomillo de vedella Strogonoff i arròs basmati,Julivert fresc 3g,Julivert fresc,3,grams
